# Machine Learning in pyspark

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Missing').getOrCreate()

24/03/18 14:31:07 WARN Utils: Your hostname, htetaunglynn resolves to a loopback address: 127.0.1.1; using 192.168.61.202 instead (on interface wlo1)
24/03/18 14:31:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/18 14:31:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/18 14:31:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/18 14:31:08 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/03/18 14:31:08 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [6]:
training = spark.read.csv('test1.csv', header=True, inferSchema=True)
training.show()

+-----+---+----------+-------+
| name|age|experience| salary|
+-----+---+----------+-------+
| Htet| 21|         5| 600000|
| Aung| 29|         4| 300000|
| Lynn| 30|         9|1000000|
|  Zaw| 25|         5| 400000|
|   Su| 31|         8| 800000|
|Thida| 26|         7|1200000|
+-----+---+----------+-------+



In [8]:
training.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: integer (nullable = true)
 |-- salary: integer (nullable = true)



In [9]:
training.columns

['name', 'age', 'experience', 'salary']

**[age, experience]----> new feature ----> independent feature**

In [10]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=['age','experience'],
                                   outputCol='independent_features')

In [11]:
output = featureassembler.transform(training)

In [12]:
output.show()

+-----+---+----------+-------+--------------------+
| name|age|experience| salary|independent_features|
+-----+---+----------+-------+--------------------+
| Htet| 21|         5| 600000|          [21.0,5.0]|
| Aung| 29|         4| 300000|          [29.0,4.0]|
| Lynn| 30|         9|1000000|          [30.0,9.0]|
|  Zaw| 25|         5| 400000|          [25.0,5.0]|
|   Su| 31|         8| 800000|          [31.0,8.0]|
|Thida| 26|         7|1200000|          [26.0,7.0]|
+-----+---+----------+-------+--------------------+



In [13]:
output.columns

['name', 'age', 'experience', 'salary', 'independent_features']

In [14]:
final_data = output.select('independent_features', 'salary')
final_data.show()

+--------------------+-------+
|independent_features| salary|
+--------------------+-------+
|          [21.0,5.0]| 600000|
|          [29.0,4.0]| 300000|
|          [30.0,9.0]|1000000|
|          [25.0,5.0]| 400000|
|          [31.0,8.0]| 800000|
|          [26.0,7.0]|1200000|
+--------------------+-------+



In [22]:
from pyspark.ml.regression import LinearRegression

# train-test-split
train_data, test_data = final_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='independent_features', 
                           labelCol='salary')

regressor = regressor.fit(train_data)

24/03/18 14:55:17 WARN Instrumentation: [db4f5576] regParam is zero, which might cause numerical instability and overfitting.


In [23]:
# coefficients
regressor.coefficients

DenseVector([-32756.0241, 203765.0602])

In [24]:
# intercepts
regressor.intercept

342921.68674699427

In [25]:
# prediction
pred_results = regressor.evaluate(test_data)
pred_results.predictions.show()

+--------------------+-------+-----------------+
|independent_features| salary|       prediction|
+--------------------+-------+-----------------+
|          [30.0,9.0]|1000000|1194126.506024098|
+--------------------+-------+-----------------+



In [26]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(194126.50602409802, 37685100341.12417)